#Entrenamiento del modelo RoBERTa

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl 

You will need to setup git, adapt your email and name in the following cell.

In [ ]:
!git config --global user.email "luisangel.motta@alumnos.viu.es"
!git config --global user.name "AngelMV97"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset
# Cargue del dataset
raw_datasets = load_dataset("LuangMV97/EmpathetiCounseling_Dataset")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/47986 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12014 [00:00<?, ? examples/s]

In [ ]:
for key in raw_datasets["train"][0]:
    print(f"{key.upper()}: {raw_datasets['train'][0][key]}")

INPUT: My friend just told me she likes me as more than a friend.
LABEL: That is a tough situation. You don't want to mess up a great friendship and you also don't want to hurt her feelings either if you don't feel the same. 


In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM, DataCollatorForLanguageModeling

context_length = 512
checkpoint = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(checkpoint)
model = RobertaForMaskedLM.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
def tokenize_function(example):
    return tokenizer(text=example["input"],
                     text_target=example["label"],
                     truncation=True,
                     padding='max_length',
                     max_length=context_length,
                     return_length=True)
# Tokenización del conjunto de datos
tokenizer.pad_token = tokenizer.eos_token
tokenized_datasets = raw_datasets.map(tokenize_function,
                                      batched=True,
                                      remove_columns=raw_datasets["train"].column_names)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=True)

Map:   0%|          | 0/47986 [00:00<?, ? examples/s]

Map:   0%|          | 0/12014 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'length', 'attention_mask', 'labels'],
        num_rows: 47986
    })
    test: Dataset({
        features: ['input_ids', 'length', 'attention_mask', 'labels'],
        num_rows: 12014
    })
})

In [ ]:
out = data_collator([tokenized_datasets["train"][i] for i in range(5)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

input_ids shape: torch.Size([5, 512])
length shape: torch.Size([5])
attention_mask shape: torch.Size([5, 512])
labels shape: torch.Size([5, 512])


Instancia el Entrenador

In [ ]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="New_RoBERTa_EmpAI_final",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    auto_find_batch_size=True,
    gradient_accumulation_steps=8,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.1,
    learning_rate=1e-4,
    warmup_steps=10_000,
    fp16=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Pruebas de entrenamiento del modelo

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
0,1.443600,1.194924
1,1.294500,1.098642
2,1.210800,1.095379
3,1.180700,1.116917
4,1.143500,1.123347


TrainOutput(global_step=3745, training_loss=1.2325162961422522, metrics={'train_runtime': 4638.0481, 'train_samples_per_second': 51.731, 'train_steps_per_second': 0.807, 'total_flos': 6.506447966082662e+16, 'train_loss': 1.2325162961422522, 'epoch': 4.99})

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1712691221.9d10d5594700.1360.0:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/LuangMV97/New_RoBERTa_EmpAI_final/commit/d3b311bec624afa77ab933b0f0d686582bfd37d6', commit_message='End of training', commit_description='', oid='d3b311bec624afa77ab933b0f0d686582bfd37d6', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.train() #weight_decay=0.1, learning_rate=1e-4, warmup_sterps=10_000 RoBERTa_EmpAI_final

Epoch,Training Loss,Validation Loss
1,12.692100,4.906155
2,0.377700,0.080903
3,0.077900,0.071495
4,0.070400,0.069692
5,0.068800,0.070380


TrainOutput(global_step=3750, training_loss=2.4424086278279624, metrics={'train_runtime': 4680.6633, 'train_samples_per_second': 51.273, 'train_steps_per_second': 0.801, 'total_flos': 6.318107853640704e+16, 'train_loss': 2.4424086278279624, 'epoch': 5.0})

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 1.07


In [ ]:
eval_results

{'eval_loss': 0.07003416866064072,
 'eval_runtime': 87.6316,
 'eval_samples_per_second': 136.96,
 'eval_steps_per_second': 17.129,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/LuangMV97/RoBERTa_EmpAI_final/commit/f46f0fe4234038ad7a1215b2ee4c934e516568dd', commit_message='End of training', commit_description='', oid='f46f0fe4234038ad7a1215b2ee4c934e516568dd', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.train() #weight_decay=0.1, learning_rate=1e-5, warmup_sterps=500 RoBERTa_EmpTalk_Prueba

Epoch,Training Loss,Validation Loss
0,No log,7.425834
2,4.901900,0.087168
4,0.403600,0.076457


Epoch,Training Loss,Validation Loss
0,No log,7.425834
2,4.901900,0.087168
4,0.080200,0.075440


TrainOutput(global_step=2415, training_loss=3.7459706292636152, metrics={'train_runtime': 3051.9735, 'train_samples_per_second': 50.684, 'train_steps_per_second': 0.791, 'total_flos': 4.068293857405747e+16, 'train_loss': 3.7459706292636152, 'epoch': 4.99})

In [ ]:
trainer.train(resume_from_checkpoint=True)

There were missing keys in the checkpoint model loaded: ['lm_head.decoder.weight', 'lm_head.decoder.bias'].


Epoch,Training Loss,Validation Loss
5,0.073700,0.070847
7,0.071200,0.070178
9,0.069800,0.069570


TrainOutput(global_step=4830, training_loss=0.0357159656027089, metrics={'train_runtime': 3046.3138, 'train_samples_per_second': 101.556, 'train_steps_per_second': 1.586, 'total_flos': 8.136587714811494e+16, 'train_loss': 0.0357159656027089, 'epoch': 9.99})

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 1.07


In [ ]:
eval_results

{'eval_loss': 0.07068553566932678,
 'eval_runtime': 56.2572,
 'eval_samples_per_second': 137.511,
 'eval_steps_per_second': 17.189,
 'epoch': 9.99}

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1712048727.cddba71158cf.1161.3:   0%|          | 0.00/359 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/LuangMV97/RoBERTa_EmpAI_FineTuned/commit/65155065b9408af51282a401333da82d406bdba7', commit_message='End of training', commit_description='', oid='65155065b9408af51282a401333da82d406bdba7', pr_url=None, pr_revision=None, pr_num=None)